In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkAssignmentLevel2").getOrCreate()

In [3]:
# Load in data
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

In [4]:
# Filter for greater than 25 total votes
total_votes = df.filter(df['total_votes']> 25)
total_votes.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48701722| RBEMQ29WJBHYG|B00YO3UYXW|     913911909|Jensen MCR-100 Ca...|Mobile_Electronics|          5|          164|        168|   N|                Y|  I LOVE my recorder|I LOVE my recorde...| 2015-08-31|
|         US|   40963441|R2JK5Y8D5MXAGP|B00SU7B0JK|      82850235|iXCC Multi pack L...|Mobile_Electronics|      

In [5]:
# Filter for greater than 50% helpful vote percentage
most_helpful_votes = df.filter(df['helpful_votes']> 0.5)
most_helpful_votes.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48701722| RBEMQ29WJBHYG|B00YO3UYXW|     913911909|Jensen MCR-100 Ca...|Mobile_Electronics|          5|          164|        168|   N|                Y|  I LOVE my recorder|I LOVE my recorde...| 2015-08-31|
|         US|    2990247|R19VVIUT4BZCMT|B011W26BU2|     938399424|Bluetooth Speaker...|Mobile_Electronics|      

In [6]:
# Paid reviews
paid_reviews = df.filter(df['vine']== 'Y')
paid_reviews.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   52433849|R147HZ0JHN3NE0|B00EP0SIOG|     628459233|Orbotix Sphero 2....|Mobile_Electronics|          5|            2|          5|   Y|                N|A Cool Toy, Cat R...|Unless you are a ...| 2013-11-27|
|         US|   14220975|R323MJVCE76RHX|B00EP0SIOG|     628459233|Orbotix Sphero 2....|Mobile_Electronics|      

In [7]:
# Unpaid reviews
unpaid_reviews = df.filter(df['vine']== 'N')
unpaid_reviews.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

In [8]:
paid_five_star_number = paid_reviews[paid_reviews['star_rating']== 5].count()
paid_number = paid_reviews.count()
percentage_five_star_vine = float(paid_five_star_number) / float(paid_number)


print(f'Paid reviews {paid_number}')
print(f'Paid five star reviews {paid_five_star_number}')
print(f'Percantage of paid reviews that are five stars {percentage_five_star_vine * 100}%')

Paid reviews 18
Paid five star reviews 6
Percantage of paid reviews that are five stars 33.33333333333333%


In [9]:
unpaid_five_star_number = unpaid_reviews[unpaid_reviews['star_rating']== 5].count()
unpaid_number = unpaid_reviews.count()
percentage_five_star_non_vine = float(unpaid_five_star_number) / float(unpaid_number)



print(f'Unpaid reviews {unpaid_number}')
print(f'Unpaid five star reviews {unpaid_five_star_number}')
print(f'Percantage of unpaid reviews that are five stars {percentage_five_star_non_vine * 100}%')

Unpaid reviews 104957
Unpaid five star reviews 52249
Percantage of unpaid reviews that are five stars 49.78133902455291%



33.3% of five-star reviews are among reviewers that are not part of the Vine program, whereas 49.8% of five-star reviews are from reviewers that are. This shows an incline in bias for positive reviews from Vine members, likely due to reciprocated advantages, like points, rewards, etc.


